In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/TER_MovieGraph"

In [ ]:
frame_path = "/content/drive/MyDrive/TER_MovieGraph/Frames/"

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-dw8_5oll
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-dw8_5oll
  Resolved https://github.com/facebookresearch/detectron2.git to commit 38af375052d3ae7331141bc1a22cfa2713b02987
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filena

In [ ]:
!pip install click==7.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 KB 3.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.3 requires click>=8.0, but you have click 7.1.2 which is incompatible.
black 23.1.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.


In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.13 ; cuda:  cu116


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = "cuda"
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = "cpu"

No GPU available, using the CPU instead.


In [ ]:
import detectron2
!nvcc --version
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
detectron2: 0.6


In [ ]:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, glob, requests
from google.colab.patches import cv2_imshow
from pprint import pprint

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# creating the custom predictor extracting the ROI information from Faster R-CNN

from detectron2.modeling.roi_heads.roi_heads import ImageList
class MyPredictor(DefaultPredictor):
  def __init__(self, cfg):
    super(MyPredictor, self).__init__(cfg)
  def __call__(self, original_image):
    with torch.no_grad():  #
      # Apply pre-processing to image.
      if self.input_format == "RGB":
          # whether the model expects BGR inputs or RGB
          original_image = original_image[:, :, ::-1]
      height, width = original_image.shape[:2]

      image = self.aug.get_transform(original_image).apply_image(original_image)
      image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))

      inputs = {"image": image, "height": height, "width": width}

      images = self.model.preprocess_image([inputs])

      features = self.model.backbone(images.tensor)
      
      proposals, _ = self.model.proposal_generator(images, features)
      instances, _ = self.model.roi_heads(images, features, proposals)
      
      features = [features[f] for f in self.model.roi_heads.in_features]
      features = self.model.roi_heads.box_pooler(features, [x.pred_boxes for x in instances])
      features = self.model.roi_heads.box_head(features)
      return instances, features


In [ ]:
# applying custom predictor
cfg = get_cfg()
cfg.MODEL.DEVICE=device
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
predictor = MyPredictor(cfg)

[03/01 10:47:19 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl ...


model_final_f6e8b1.pkl: 243MB [00:07, 32.7MB/s]                           


In [ ]:
from detectron2.modeling import build_model

model = build_model(cfg)
# model.eval()

In [ ]:
# extracting coco classes the Faster-RCNN is trained on
coco_classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
pprint(coco_classes)

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']


In [ ]:
# vidvrd classes
vidvrd_classes = ['__background__',  # always index 0
            'airplane', 'antelope', 'bear', 'bicycle',
            'bird', 'bus', 'car', 'cattle',
            'dog', 'domestic_cat', 'elephant', 'fox',
            'giant_panda', 'hamster', 'horse', 'lion',
            'lizard', 'monkey', 'motorcycle', 'rabbit',
            'red_panda', 'sheep', 'snake', 'squirrel',
            'tiger', 'train', 'turtle', 'watercraft',
            'whale', 'zebra',                               
            'ball','frisbee','sofa','skateboard','person']
for i, line in enumerate(vidvrd_classes):
    print(i , line)
print(len(vidvrd_classes))
pprint(vidvrd_classes)

0 __background__
1 airplane
2 antelope
3 bear
4 bicycle
5 bird
6 bus
7 car
8 cattle
9 dog
10 domestic_cat
11 elephant
12 fox
13 giant_panda
14 hamster
15 horse
16 lion
17 lizard
18 monkey
19 motorcycle
20 rabbit
21 red_panda
22 sheep
23 snake
24 squirrel
25 tiger
26 train
27 turtle
28 watercraft
29 whale
30 zebra
31 ball
32 frisbee
33 sofa
34 skateboard
35 person
36
['__background__',
 'airplane',
 'antelope',
 'bear',
 'bicycle',
 'bird',
 'bus',
 'car',
 'cattle',
 'dog',
 'domestic_cat',
 'elephant',
 'fox',
 'giant_panda',
 'hamster',
 'horse',
 'lion',
 'lizard',
 'monkey',
 'motorcycle',
 'rabbit',
 'red_panda',
 'sheep',
 'snake',
 'squirrel',
 'tiger',
 'train',
 'turtle',
 'watercraft',
 'whale',
 'zebra',
 'ball',
 'frisbee',
 'sofa',
 'skateboard',
 'person']


In [ ]:
# mapping of Vidvrd classes with coco classes
coco_to_vidvrd_map = {
    'sports ball': 'ball',
    'cow': 'cattle',
    'boat': 'watercraft',
    'couch': 'sofa',
    'cat': 'domestic_cat',
}

In [ ]:
!pip install imageio --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


In [ ]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install imageio[ffmpeg]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 37.9 MB/s eta 0:00:00


In [ ]:
# creating frames from video scene 
import imageio.v3 as iio
frames_dict = {}

for idx, frame in enumerate(iio.imiter("/content/drive/MyDrive/TER_MovieGraph/cuckoo_nest_scene.mp4")):

#  cv2.imwrite(os.path.join(frame_path, f'frame_{idx}.png'), frame) # uncomment this line if you want to save frames to a folder

# saving frames to a dictionary for further processig, saving width and heigth
  frames_dict[idx] = frame
  if idx == 0:
    width = frame.shape[1]
    height = frame.shape[0]  
   #create a dictionary of frames

print(frames_dict[0])

[[[128 115 103]
  [128 115 103]
  [128 115 103]
  ...
  [143 130 116]
  [143 130 116]
  [143 130 116]]

 [[128 115 103]
  [128 115 103]
  [128 115 103]
  ...
  [143 130 116]
  [143 130 116]
  [143 130 116]]

 [[127 114 102]
  [127 114 102]
  [127 114 102]
  ...
  [143 130 116]
  [143 130 116]
  [143 130 116]]

 ...

 [[ 34  24  18]
  [ 34  24  18]
  [ 34  24  18]
  ...
  [ 69  59  48]
  [ 68  58  47]
  [ 67  57  46]]

 [[ 32  22  16]
  [ 32  22  16]
  [ 33  23  17]
  ...
  [ 69  59  48]
  [ 68  58  47]
  [ 67  57  46]]

 [[ 31  21  15]
  [ 31  21  15]
  [ 32  22  16]
  ...
  [ 69  59  48]
  [ 68  58  47]
  [ 67  57  46]]]


In [ ]:
frames_dict[0][1]

array([[128, 115, 103],
       [128, 115, 103],
       [128, 115, 103],
       ...,
       [143, 130, 116],
       [143, 130, 116],
       [143, 130, 116]], dtype=uint8)

In [ ]:
# uncomment the code below if you want to read frames from the destination where you saved it
# import glob
# for im in frame_path:
#  images = [cv2.imread(file) for file in glob.glob(f'{frame_path}*.png')]
#  cv2_imshow(images[0])

In [ ]:
# function decoding coco_class_id to vidvrd_class_id
def coco_to_vidvrd(coco_class_id):
  coco_class_name = coco_classes[coco_class_id]
  if coco_class_name in vidvrd_classes:
    vidvrd_class_id = vidvrd_classes.index(coco_class_name)
  elif coco_class_name in coco_to_vidvrd_map:
    vidvrd_class_id = vidvrd_classes.index(coco_to_vidvrd_map[coco_class_name])
  else:
    vidvrd_class_id = None
  return vidvrd_class_id

In [ ]:
# checking the format of data needed for deepSORT to process
import pickle

with open('/content/drive/MyDrive/TER_MovieGraph/VidORval_freq1_0024.pkl', 'rb') as fp:
  p = pickle.load(fp)

In [ ]:
print(p.keys())

dict_keys(['0024_5328004991', '0024_6542914065', '0024_6977613963'])


In [ ]:
print(p['0024_5328004991'][0])

{'video_name': '0024_5328004991', 'frame_id': 0, 'width_height': (640, 480), 'bboxes': array([], shape=(0, 4), dtype=float32), 'scores': array([], dtype=float32), 'labels': array([], dtype=int64), 'roifeats': array([], shape=(0, 1024), dtype=float32)}


In [ ]:
# creating the outputs of the predictor of Faster-RCNN in a format that deepSORT could process

all_outputs = []
for iframe, im in frames_dict.items():
    all_outputs.append({
        'video_name': 'test',
        'frame_id': 0,
        'width_height': (width, height),
        'bboxes': [],
        'scores': [],
        'labels': [],
        'roifeats': []
    })
    instances, features = predictor(im)
    instances = instances[0]
    features = features.cpu().numpy()

    bboxes = instances.pred_boxes.tensor.cpu().numpy()

    for i, coco_class_id in reversed(list(enumerate(instances.pred_classes))):
      # print(i)
      if coco_to_vidvrd(coco_class_id) is None:
          bboxes = np.concatenate((
              bboxes[:i, :],
              bboxes[i + 1:, :],
          ))
          features = np.concatenate((
              features[:i, :],
              features[i + 1:, :],
          ))
      # print(bboxes)

    all_outputs[-1]['bboxes'] = bboxes
    all_outputs[-1]['roifeats'] = features

    for i, coco_class_id in enumerate(instances.pred_classes):
        vidvrd_class_id = coco_to_vidvrd(coco_class_id)
        if vidvrd_class_id is not None:
            all_outputs[-1]['labels'].append(vidvrd_class_id)
            all_outputs[-1]['scores'].append(instances.scores[i].item())
    all_outputs[-1]['frame_id'] = iframe

    print(all_outputs[-1])


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


{'video_name': 'test', 'frame_id': 0, 'width_height': (1920, 1080), 'bboxes': array([[ 321.35422  ,  284.3483   ,  380.42203  ,  451.13406  ],
       [  98.41571  ,  267.25824  ,  170.40747  ,  379.04938  ],
       [ 817.8874   ,  197.82812  , 1328.0061   ,  706.47974  ],
       [   2.1140175,  298.9925   ,   93.22377  ,  457.16852  ]],
      dtype=float32), 'scores': [0.9980204105377197, 0.827505350112915, 0.40922170877456665, 0.05579756200313568], 'labels': [35, 35, 35, 35], 'roifeats': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.5877627 , 0.53730804,
        0.        ],
       [0.        , 0.03017659, 0.        , ..., 1.1763589 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.6598253 , 0.        ,
        0.        ]], dtype=float32)}
{'video_name': 'test', 'frame_id': 1, 'width_height': (1920, 1080), 'bboxes': array([[ 321.37454,  288.86255,  38

In [ ]:
all_outputs

[{'video_name': 'test',
  'frame_id': 0,
  'width_height': (1920, 1080),
  'bboxes': array([[ 321.35422  ,  284.3483   ,  380.42203  ,  451.13406  ],
         [  98.41571  ,  267.25824  ,  170.40747  ,  379.04938  ],
         [ 817.8874   ,  197.82812  , 1328.0061   ,  706.47974  ],
         [   2.1140175,  298.9925   ,   93.22377  ,  457.16852  ]],
        dtype=float32),
  'scores': [0.9980204105377197,
   0.827505350112915,
   0.40922170877456665,
   0.05579756200313568],
  'labels': [35, 35, 35, 35],
  'roifeats': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , ..., 0.5877627 , 0.53730804,
          0.        ],
         [0.        , 0.03017659, 0.        , ..., 1.1763589 , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , ..., 0.6598253 , 0.        ,
          0.        ]], dtype=float32)},
 {'video_name': 'test',
  'frame_id': 1,
  'width_height': (1920, 10

In [ ]:
# saving outputs
import os.path
import pickle

with open(os.path.join(path, 'all_outputs.pkl'), 'wb') as fp:
    pickle.dump(all_outputs, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# we created the list, but we need a dictionary. There could be several video files
type(all_outputs)

list

In [ ]:
key = "test"
output = {}
output[key] = all_outputs

In [ ]:
type(output)

dict

In [ ]:
output['test'][0]

{'video_name': 'test',
 'frame_id': 0,
 'width_height': (1920, 1080),
 'bboxes': array([[ 321.35422  ,  284.3483   ,  380.42203  ,  451.13406  ],
        [  98.41571  ,  267.25824  ,  170.40747  ,  379.04938  ],
        [ 817.8874   ,  197.82812  , 1328.0061   ,  706.47974  ],
        [   2.1140175,  298.9925   ,   93.22377  ,  457.16852  ]],
       dtype=float32),
 'scores': [0.9980204105377197,
  0.827505350112915,
  0.40922170877456665,
  0.05579756200313568],
 'labels': [35, 35, 35, 35],
 'roifeats': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.5877627 , 0.53730804,
         0.        ],
        [0.        , 0.03017659, 0.        , ..., 1.1763589 , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.6598253 , 0.        ,
         0.        ]], dtype=float32)}

In [ ]:
with open(os.path.join(path, 'output.pkl'), 'wb') as fp:
    pickle.dump(output, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# this cell does not work on collab, but it works locally when you clone the repository
# from here https://github.com/Dawn-LX/VidVRD-tracklets/blob/master/deepSORT/deepSORT_tracking_v2.py and edit the file 
# tolist() needs to be deleted at the lines 
#        scores = proposals["scores"].tolist()
#        labels = proposals["labels"].tolist()
# because in outputs dictionary file scores and labels are already lists.

import os
import pickle
import numpy as np
import random
import cv2
from tqdm import tqdm
from deep_sort_app_v2 import run

### default args:
nms_max_overlap = 1.0
min_detection_height = 0
max_cosine_distance = 0.2
nn_budget = None
display = False
frameRate = 25

### modified args:
min_confidence = 0.3
max_miss = 60


all_frames_path = None
proposal_result_path = "/content/drive/MyDrive/TER_MovieGraph/all_outputs.pkl"   # original frame-level proposal results   

save_dir = "/content/drive/MyDrive/TER_MovieGraph/NPY"#

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
print("save_dir==",save_dir)

with open(proposal_result_path,'rb') as f:
    proposal_results = pickle.load(f)

video_name_list = list(proposal_results.keys())


for video_name in tqdm(video_name_list):  
    proposals_list = proposal_results[video_name]
    output_file = os.path.join(save_dir, video_name+".npy")
    if os.path.exists(output_file):
        continue
    ## step1 convert to deepSORT_format, refer to original script 'generate_deep_sort_format_v2.py'
    proposals_per_video = []
    for frame_id, proposals in enumerate(proposals_list):
        assert video_name == proposals["video_name"]
        assert frame_id == proposals["frame_id"], "frame_id:{} != {}".format(frame_id,proposals["frame_id"])
        bboxes = proposals["bboxes"]                # refer to  `MEGA/mega_boxfeatures/cvt_proposal_result.py`
        roifeats = proposals["roifeats"]
        scores = proposals["scores"]
        labels = proposals["labels"]
        width,height = proposals["width_height"]
        
        for i, bbox in enumerate(bboxes):
            vector_per_box = []
            vector_per_box += [frame_id,-1]
            vector_per_box += bbox.tolist()           # format="xywh"
            vector_per_box += [scores[i]]
            vector_per_box += [labels[i]]
            vector_per_box += [-1,-1]
            vector_per_box += roifeats[i,:].tolist()
            proposals_per_video.append(vector_per_box)
    
    proposals_per_video = np.array(proposals_per_video)

    ## step2. gather_sequence_info, refer to the original function in `gather_sequence_info` in deep_sort_app_v2.py
    if all_frames_path != None:
        video_frames_dir = os.path.join(all_frames_path,video_name)  # .JPEG files for all the frames of the video
        image_filenames = {
            int(f.split(".")[0]): os.path.join(video_frames_dir, f)
            for f in os.listdir(video_frames_dir)}
    else:
        image_filenames = None
    detections = proposals_per_video
    image_size = (height,width)
    min_frame_idx = int(detections[:, 0].min())
    max_frame_idx = int(detections[:, 0].max())

    feature_dim = detections.shape[1] - 10 if detections is not None else 0   
    seq_info = {
        # "sequence_name": os.path.basename(video_frames_dir),
        "image_filenames": image_filenames,
        "detections": detections,
        "groundtruth": None,
        "image_size": image_size,
        "min_frame_idx": min_frame_idx,
        "max_frame_idx": max_frame_idx,
        "feature_dim": feature_dim,
        "update_ms": 1000 / int(frameRate)
    }
    ## step3. run tracking, use the function `run` from deep_sort_app_v2.py
    run(
        sequence_dir=None,
        detection_file=None,
        min_confidence = min_confidence,
        output_file = output_file,
        nms_max_overlap = nms_max_overlap,
        min_detection_height= min_detection_height,
        max_cosine_distance = max_cosine_distance,
        nn_budget = nn_budget,
        display = display,
        max_miss = max_miss,
        seq_info=seq_info
    )